# Setup

### Airflow

1 - Atualizar a lista de pacotes do APT:

```bash
sudo apt-get update
```

2 - Instalar pacotes necessários para adicionar um novo repositório via HTTPS:

```bash
sudo apt-get install ca-certificates curl gnupg lsb-release
```

3 - Criar diretório para armazenar as chaves de repositórios

```bash
sudo mkdir -m 0755 -p /etc/apt/keyrings
```

4 - Adicionar a chave GPG do repositório do Docker:

```bash
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
```

5 - Adicionar o repositório do Docker às fontes do APT:

```bash
echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
```

6 - Atualiza a lista de pacotes após adicionar o novo repositório do Docker

```bash
sudo apt-get update
```

7 - Instalar o Docker e componentes

```bash
sudo apt-get install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin
```

8 - Baixar o arquivo docker-compose.yaml do Airflow:

```bash
curl -LfO 'https://airflow.apache.org/docs/apache-airflow/stable/docker-compose.yaml'
```

9 - Criar diretórios para DAGs, logs e plugins:

```bash
mkdir -p ./dags ./logs ./plugins
```

10 - Criar um arquivo .env com o UID do usuário usado pelo docker para permissões e acessos para o a interface airflow (inserir senha padrão)

```bash
echo -e "AIRFLOW_UID=$(id -u)\n_AIRFLOW_WWW_USER_USERNAME=postech-mlengineering\n_AIRFLOW_WWW_USER_PASSWORD=<senha>" > .env
```

11 - Iniciar o airflow

```bash
sudo docker compose up airflow-init
```

12 - Subir o Airflow em modo desacoplado

```bash
sudo docker compose up -d
```

13 - Configurar conexão na interface Airflow:

Em Admin > Connections > Add Connection, configure:

<img src='img/airflow_connection.png'>


### PostgresSQL


1 - Atualizar a lista de pacotes do APT e instalar as dependências:

```bash
sudo apt-get update

sudo apt install curl ca-certificates gnupg
```

2 - Adicionar o repositório APT do PostgreSQL:

```bash
curl -fsSL https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo gpg --dearmor -o /etc/apt/keyrings/postgresql.gpg

echo "deb [signed-by=/etc/apt/keyrings/postgresql.gpg] http://apt.postgresql.org/pub/repos/apt jammy-pgdg main" | sudo tee /etc/apt/sources.list.d/pgdg.list > /dev/null
```

3 - Atualizar a lista de pacotes e enstalar o PostgreSQL 17:

```bash
sudo apt update

sudo apt install postgresql-17 postgresql-client-17 -y
```

5 - Verificar o status do serviço:

```bash
sudo systemctl status postgresql
```

6 - Acessar o console do PostgreSQL e criar usuário (inserir senha padrão):

```bash
sudo -u postgres psql

CREATE USER "postech-mlengineering" WITH PASSWORD '<senha>' CREATEDB;
```

7 - Criar banco de dados:

```bash
CREATE DATABASE bookstoscrape OWNER "postech-mlengineering";

GRANT ALL PRIVILEGES ON DATABASE bookstoscrape TO "postech-mlengineering";
```

8 - Configurar acesso remoto:

```bash
sudo nano /etc/postgresql/17/main/postgresql.conf
```

Descomente o parâmetro "listen_addresses" e defina '*':

```conf
listen_addresses = '*'
```

9 - Configurar autenticação:

```bash
sudo nano /etc/postgresql/17/main/pg_hba.conf
```

Adicione:

```conf
# TYPE  DATABASE        USER            ADDRESS                 METHOD
host    bookstoscrape   postech-mlengineering   0.0.0.0/0               md5
```

10 - Reiniciar o serviço:

```bash
sudo systemctl restart postgresql
```